In [ ]:
# install huggingface
!pip install transformers
!pip install emoji

     |████████████████████████████████| 2.3MB 28.4MB/s 
     |████████████████████████████████| 3.3MB 50.8MB/s 
     |████████████████████████████████| 901kB 40.2MB/s 
     |████████████████████████████████| 133kB 25.1MB/s 


In [ ]:
# import HuggingFace models
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

# import csv to deal with .csv data files
import csv

# import pandas to just be able to visualise our data files
import pandas as pd

# classic shit
import numpy as np
import torch

# for getting right format for data
from torch.utils.data import Dataset, DataLoader

# for computing metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# import for preprocess the data
import random
import emoji
import string
import re

In [ ]:
%env WANDB_DISABLED=true

# Fetch the dataset from github and shuffle the rows. This because all the label 1 are in the same part of the dataset.

In [ ]:
!git clone https://github.com/MeliLuca/Natural_Language_Processing

In [ ]:
# open train data
with open("/content/Natural_Language_Processing/sexism-dataset.csv", "r", encoding="utf8") as f:
    data = [{k: v for k, v in row.items()} for row in csv.DictReader(f, skipinitialspace=True)] 

random.shuffle(data)

In [ ]:
train_data = data[:9000]
test_data = data[9000:]

# Start to define the preprocess function and separate Label from text

In [ ]:
def clean_tweet(tweet):
    
    # tokenize tweet
    tokenized_tweet = " ".join(tokenizer.tokenize(tweet))
    # remove unprintable characters (ð\x9f\x98\x81ð\x9f\x98\x81 for ex)
    printable_tweet = ''.join(filter(lambda x: x in set(string.printable), tokenized_tweet))
    
    # replace links (https://www. for ex) HTTPURL
    linkless_tweet = re.sub(r"http\S+", "HTTPURL", printable_tweet)
    
    # remove retweets (RT)
    retweetless_tweet = re.sub(r"RT", "", linkless_tweet)
    
    # replace mentions (@user for example) with @USER
    mentionless_tweet = re.sub(r"@[\S]+", "@USER", retweetless_tweet)
    
    # remove other odd stuff (&... for ex)
    other = re.sub(r"&[\S]+", "", mentionless_tweet)
    
    # replace emojis with str
    emojiless_tweet = emoji.demojize(other)
    
    # remove unwanted whitespace (ex "Hello        XWorld" => "hell world")
    whitespaceless = re.sub(r'\s+', ' ', emojiless_tweet)
    
    return whitespaceless

In [ ]:
train_data_tweets = [clean_tweet(row["text"]) for row in train_data] 
train_data_labels = [int(row["label"]) for row in train_data]

test_data_tweets = [clean_tweet(row["text"]) for row in test_data]
test_data_labels = [int(row["label"]) for row in test_data]

In [ ]:
train_d = {'text': [clean_tweet(row["text"]) for row in train_data] , 'label': [int(row["label"]) for row in train_data]}
test_d = {'text': [clean_tweet(row["text"]) for row in test_data] , 'label': [int(row["label"]) for row in test_data]}

# Save the clean datasets

In [ ]:
train_frame = pd.DataFrame(train_d)
test_frame = pd.DataFrame(test_d)

train_frame.to_csv('clean_train_set.csv', header=True)
test_frame.to_csv('clean_test_set.csv', header=True)